## Imports & Environment Setup

In [8]:
import numpy as np
import tensorflow as tf
import scipy.sparse
import os
import time
import sys
import argparse

from pathlib import Path


## Define Hyperparameters and Dataset Configuration

In [9]:
DATASET = 'yelp2018'
DATA_ROOT = '../data/ml-1m' 

class Args:
    dataset = DATASET
    verbose = 10
    batch_size = 512
    epochs = 501
    embed_size = 64
    lr = 0.05
    dropout = 1.0
    negative_weight = 0.5
    topK = [5, 10, 20]

args = Args()
random_seed = 2019
np.random.seed(random_seed)


## Define the LoadData Class

In [10]:
class LoadData(object):

    def __init__(self, DATA_ROOT):
        self.trainfile = os.path.join(DATA_ROOT, 'train.csv')
        self.testfile = os.path.join(DATA_ROOT, 'test.csv')
        self.user_field_M, self.item_field_M = self.get_length()
        print("user_field_M", self.user_field_M)
        print("item_field_M", self.item_field_M)
        print("field_M", self.user_field_M + self.item_field_M)
        self.item_bind_M = self.bind_item()  # assaign a userID for a specific user-context
        self.user_bind_M = self.bind_user()  # assaign a itemID for a specific item-feature
        print("item_bind_M", len(self.binded_items.values()))
        print("user_bind_M", len(self.binded_users.values()))
        self.item_map_list=[]
        for itemid in self.item_map.keys():
            self.item_map_list.append([int(feature) for feature in self.item_map[itemid].strip().split('-')[0:]])
        self.item_map_list.append([int(feature) for feature in self.item_map[0].strip().split('-')[0:]])
        self.user_positive_list = self.get_positive_list(self.trainfile)  # userID positive itemID
        self.Train_data, self.Test_data = self.construct_data()
        self.user_train, self.item_train = self.get_train_instances()
        self.user_test=self.get_test()


    def get_length(self):
        '''
        map the user fields in all files, kept in self.user_fields dictionary
        :return:
        '''
        length_user = 0
        length_item = 0
        f = open(self.trainfile)
        line = f.readline()
        while line:
            user_features = line.strip().split(',')[0].split('-')
            item_features = line.strip().split(',')[1].split('-')
            for user_feature in user_features:
                feature = int(user_feature)
                if feature > length_user:
                    length_user = feature
            for item_feature in item_features:
                feature = int(item_feature)
                if feature > length_item:
                    length_item = feature
            line = f.readline()
        f.close()
        return length_user + 1, length_item + 1

    def bind_item(self):
        '''
        Bind item and feature
        :return:
        '''
        self.binded_items = {}  # dic{feature: id}
        self.item_map = {}  # dic{id: feature}
        self.bind_i(self.trainfile)
        self.bind_i(self.testfile)
        return len(self.binded_items)

    def bind_i(self, file):
        '''
        Read a feature file and bind
        :param file: feature file
        :return:
        '''
        f = open(file)
        line = f.readline()
        i = len(self.binded_items)
        while line:
            features = line.strip().split(',')
            item_features = features[1]
            if item_features not in self.binded_items:
                self.binded_items[item_features] = i
                self.item_map[i] = item_features
                i = i + 1
            line = f.readline()
        f.close()

    def bind_user(self):
        '''
        Map the item fields in all files, kept in self.item_fields dictionary
        :return:
        '''
        self.binded_users = {}
        self.user_map={}
        self.bind_u(self.trainfile)
        self.bind_u(self.testfile)
        return len(self.binded_users)

    def bind_u(self, file):
        '''
        Read a feature file and bind
        :param file:
        :return:
        '''
        f = open(file)
        line = f.readline()
        i = len(self.binded_users)
        while line:
            features = line.strip().split(',')
            user_features = features[0]
            if user_features not in self.binded_users:
                self.binded_users[user_features] = i
                self.user_map[i]=user_features
                i = i + 1
            line = f.readline()
        f.close()

    def get_positive_list(self, file):
        '''
        Obtain positive item lists for each user
        :param file: train file
        :return:
        '''
        self.max_positive_len=0
        f = open(file)
        line = f.readline()
        user_positive_list = {}
        while line:
            features = line.strip().split(',')
            user_id = self.binded_users[features[0]]
            item_id = self.binded_items[features[1]]
            if user_id in user_positive_list:
                user_positive_list[user_id].append(item_id)
            else:
                user_positive_list[user_id] = [item_id]
            line = f.readline()
        f.close()
        for i in user_positive_list:
            if len(user_positive_list[i])>self.max_positive_len:
                self.max_positive_len=len(user_positive_list[i])
        return user_positive_list

    def get_train_instances(self):
        user_train, item_train = [], []
        for i in self.user_positive_list:
            u_train = [int(feature) for feature in self.user_map[i].strip().split('-')[0:]]
            user_train.append(u_train)
            temp=self.user_positive_list[i]
            while len(temp) < self.max_positive_len:
                temp.append(self.item_bind_M)
            item_train.append(temp)
        user_train = np.array(user_train)
        item_train = np.array(item_train)
        return user_train, item_train

    def construct_data(self):
        X_user, X_item = self.read_data(self.trainfile)
        Train_data=self.construct_dataset(X_user, X_item)
        print("# of training:", len(X_user))
        X_user, X_item = self.read_data(self.testfile)
        Test_data = self.construct_dataset(X_user, X_item)
        print("# of test:", len(X_user))

        return Train_data, Test_data

    def construct_dataset(self, X_user, X_item):

        user_id = []
        for one in X_user:
            user_id.append(self.binded_users["-".join([str(item) for item in one[0:]])])
        item_id = []
        for one in X_item:
            item_id.append(self.binded_items["-".join([str(item) for item in one[0:]])])
        count = np.ones(len(X_user))
        sparse_matrix = scipy.sparse.csr_matrix((count, (user_id, item_id)), dtype=np.int16,
                                          shape=(self.user_bind_M, self.item_bind_M))
        return sparse_matrix

    def get_test(self):
        X_user, X_item = self.read_data(self.testfile)
        return X_user



    # lists of user and item
    def read_data(self, file):
        '''
        read raw data
        :param file: data file
        :return: structured data
        '''
        # read a data file;
        f = open(file)
        X_user = []
        X_item = []
        line = f.readline()
        while line:
            features = line.strip().split(',')
            user_features = features[0].split('-')
            X_user.append([int(item) for item in user_features[0:]])
            item_features = features[1].split('-')
            X_item.append([int(item) for item in item_features[0:]])
            line = f.readline()
        f.close()
        return X_user, X_item


## Define the ENSFM Model

In [11]:
class ENSFM:
    def __init__(self, item_attribute, user_field_M, item_field_M, embedding_size, max_item_pu, args):
        self.embedding_size = embedding_size
        self.max_item_pu = max_item_pu
        self.user_field_M = user_field_M
        self.item_field_M = item_field_M
        self.weight1 = args.negative_weight
        self.item_attribute = item_attribute
        self.lambda_bilinear = [0.0, 0.0]

    def _create_placeholders(self):
        self.input_u = tf.compat.v1.placeholder(tf.int32, [None, None], name="input_u_feature")
        self.input_ur = tf.compat.v1.placeholder(tf.int32, [None, self.max_item_pu], name="input_ur")
        self.dropout_keep_prob = tf.compat.v1.placeholder(tf.float32, name="dropout_keep_prob")

    def _create_variables(self):
        self.uidW = tf.Variable(tf.random.truncated_normal(shape=[self.user_field_M, self.embedding_size],
                                                             mean=0.0, stddev=0.01), dtype=tf.float32, name="uidW")
        self.iidW = tf.Variable(tf.random.truncated_normal(shape=[self.item_field_M+1, self.embedding_size],
                                                             mean=0.0, stddev=0.01), dtype=tf.float32, name="iidW")
        self.H_i = tf.Variable(tf.constant(0.01, shape=[self.embedding_size, 1]), name="hi")
        self.H_s = tf.Variable(tf.constant(0.01, shape=[self.embedding_size, 1]), name="hs")
        self.u_bias = tf.Variable(tf.random.truncated_normal(shape=[self.user_field_M, 1],
                                                               mean=0.0, stddev=0.01), dtype=tf.float32, name="u_bias")
        self.i_bias = tf.Variable(tf.random.truncated_normal(shape=[self.item_field_M, 1],
                                                               mean=0.0, stddev=0.01), dtype=tf.float32, name="i_bias")
        self.bias = tf.Variable(tf.constant(0.0), name='bias')

    def _create_vectors(self):
        self.user_feature_emb = tf.nn.embedding_lookup(self.uidW, self.input_u)
        self.summed_user_emb = tf.reduce_sum(self.user_feature_emb, 1)

        # Apply dropout to H_i and H_s
        self.H_i = tf.nn.dropout(self.H_i, rate=1 - self.dropout_keep_prob)
        self.H_s = tf.nn.dropout(self.H_s, rate=1 - self.dropout_keep_prob)

        self.all_item_feature_emb = tf.nn.embedding_lookup(self.iidW, self.item_attribute)
        self.summed_all_item_emb = tf.reduce_sum(self.all_item_feature_emb, 1)

        # Compute cross terms for user and item
        self.user_cross = 0.5 * (tf.square(self.summed_user_emb) - tf.reduce_sum(tf.square(self.user_feature_emb), 1))
        self.item_cross = 0.5 * (tf.square(self.summed_all_item_emb) - tf.reduce_sum(tf.square(self.all_item_feature_emb), 1))

        self.user_cross_score = tf.matmul(self.user_cross, self.H_s)
        self.item_cross_score = tf.matmul(self.item_cross, self.H_s)

        self.user_bias = tf.reduce_sum(tf.nn.embedding_lookup(self.u_bias, self.input_u), 1)
        self.item_bias = tf.reduce_sum(tf.nn.embedding_lookup(self.i_bias, self.item_attribute), 1)

        self.I = tf.ones(shape=(tf.shape(self.input_u)[0], 1))
        self.p_emb = tf.concat([self.summed_user_emb, self.user_cross_score + self.user_bias + self.bias, self.I], 1)

        self.I = tf.ones(shape=(tf.shape(self.summed_all_item_emb)[0], 1))
        self.q_emb = tf.concat([self.summed_all_item_emb, self.I, self.item_cross_score + self.item_bias], 1)

        self.H_i_emb = tf.concat([self.H_i, [[1.0]], [[1.0]]], 0)

    def _create_inference(self):
        self.pos_item = tf.nn.embedding_lookup(self.q_emb, self.input_ur)
        # Filter out padding values (value == data.item_bind_M)
        self.pos_num_r = tf.cast(tf.not_equal(self.input_ur, data.item_bind_M), 'float32')
        self.pos_item = tf.einsum('ab,abc->abc', self.pos_num_r, self.pos_item)

        self.pos_r = tf.einsum('ac,abc->abc', self.p_emb, self.pos_item)
        self.pos_r = tf.einsum('ajk,kl->ajl', self.pos_r, self.H_i_emb)
        self.pos_r = tf.reshape(self.pos_r, [-1, self.max_item_pu])

    def _pre(self):
        dot = tf.einsum('ac,bc->abc', self.p_emb, self.q_emb)
        pre = tf.einsum('ajk,kl->aj', dot, self.H_i_emb)
        return pre

    def _create_loss(self):
        self.loss1 = self.weight1 * tf.reduce_sum(
            tf.reduce_sum(tf.reduce_sum(tf.einsum('ab,ac->abc', self.q_emb, self.q_emb), 0)
                          * tf.reduce_sum(tf.einsum('ab,ac->abc', self.p_emb, self.p_emb), 0)
                          * tf.matmul(self.H_i_emb, self.H_i_emb, transpose_b=True), 0), 0)
        self.loss1 += tf.reduce_sum((1.0 - self.weight1) * tf.square(self.pos_r) - 2.0 * self.pos_r)
        self.l2_loss0 = tf.nn.l2_loss(self.uidW)
        self.l2_loss1 = tf.nn.l2_loss(self.iidW)
        self.loss = self.loss1 + self.lambda_bilinear[0] * self.l2_loss0 + self.lambda_bilinear[1] * self.l2_loss1

        self.reg_loss = self.lambda_bilinear[0] * self.l2_loss0 + self.lambda_bilinear[1] * self.l2_loss1

    def _build_graph(self):
        self._create_placeholders()
        self._create_variables()
        self._create_vectors()
        self._create_inference()
        self._create_loss()
        self.pre = self._pre()


## Initialize train & evaluate

In [12]:
def train_step1(u_batch, y_batch, args):
    feed_dict = {
        deep.input_u: u_batch,
        deep.input_ur: y_batch,
        deep.dropout_keep_prob: args.dropout,
    }
    _, loss, loss1, loss2, p_emb = sess.run(
        [train_op1, deep.loss, deep.loss1, deep.reg_loss, deep.p_emb],
        feed_dict)
    return loss, loss1, loss2

def evaluate():
    eva_batch = 128
    recall_all, ndcg_all = [[] for _ in range(3)], [[] for _ in range(3)]
    user_features = data.user_test
    ll = int(len(user_features) / eva_batch) + 1

    for batch_num in range(ll):
        start_index = batch_num * eva_batch
        end_index = min((batch_num + 1) * eva_batch, len(user_features))
        u_batch = user_features[start_index:end_index]
        batch_users = end_index - start_index

        feed_dict = {
            deep.input_u: u_batch,
            deep.dropout_keep_prob: 1.0,
        }

        pre = sess.run(deep.pre, feed_dict)
        pre = np.array(pre)
        pre = np.delete(pre, -1, axis=1)

        user_id = [data.binded_users["-".join(map(str, one))] for one in u_batch]

        idx = np.zeros_like(pre, dtype=bool)
        idx[data.Train_data[user_id].nonzero()] = True
        pre[idx] = -np.inf

        for i, kj in enumerate(args.topK):
            idx_topk_part = np.argpartition(-pre, kj, 1)
            pre_bin = np.zeros_like(pre, dtype=bool)
            pre_bin[np.arange(batch_users)[:, None], idx_topk_part[:, :kj]] = True

            true_bin = np.zeros_like(pre, dtype=bool)
            true_bin[data.Test_data[user_id].nonzero()] = True

            tmp = (np.logical_and(true_bin, pre_bin).sum(axis=1)).astype(np.float32)
            recall_all[i].append(tmp / np.minimum(kj, true_bin.sum(axis=1)))

            topk_part = pre[np.arange(batch_users)[:, None], idx_topk_part[:, :kj]]
            idx_part = np.argsort(-topk_part, axis=1)
            idx_topk = idx_topk_part[np.arange(batch_users)[:, None], idx_part]

            tp = np.log(2) / np.log(np.arange(2, kj + 2))
            test_batch = data.Test_data[user_id]
            DCG = (test_batch[np.arange(batch_users)[:, None], idx_topk].toarray() * tp).sum(axis=1)
            IDCG = np.array([(tp[:min(n, kj)]).sum() for n in test_batch.getnnz(axis=1)])
            ndcg_all[i].append(DCG / IDCG)

    for i in range(3):
        recall_all[i] = np.hstack(recall_all[i])
        ndcg_all[i] = np.hstack(ndcg_all[i])
        print(f"Top {args.topK[i]} Recall: {np.mean(recall_all[i]):.4f}, NDCG: {np.mean(ndcg_all[i]):.4f}")


## Train ENSFM

In [13]:
data = LoadData(DATA_ROOT)

with tf.Graph().as_default():
    tf.compat.v1.set_random_seed(random_seed)
    session_conf = tf.compat.v1.ConfigProto()
    session_conf.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=session_conf)
    with sess.as_default():
        deep = ENSFM(data.item_map_list, data.user_field_M, data.item_field_M,
                     args.embed_size, data.max_positive_len, args)
        deep._build_graph()
        train_op1 = tf.compat.v1.train.AdagradOptimizer(
            learning_rate=args.lr, initial_accumulator_value=1e-8
        ).minimize(deep.loss)
        sess.run(tf.compat.v1.global_variables_initializer())

        evaluate()  # initial evaluation

        for epoch in range(args.epochs):
            print(f"\nEpoch {epoch}")
            shuffle_indices = np.random.permutation(len(data.user_train))
            data.user_train = data.user_train[shuffle_indices]
            data.item_train = data.item_train[shuffle_indices]

            ll = int(len(data.user_train) / args.batch_size)
            loss = np.zeros(3)

            for batch_num in range(ll):
                start_index = batch_num * args.batch_size
                end_index = min((batch_num + 1) * args.batch_size, len(data.user_train))
                u_batch = data.user_train[start_index:end_index]
                i_batch = data.item_train[start_index:end_index]
                loss1, loss2, loss3 = train_step1(u_batch, i_batch, args)
                loss += [loss1, loss2, loss3]

            print(f"loss={loss[0]/ll:.4f}, loss1={loss[1]/ll:.4f}, reg={loss[2]/ll:.4f}")

            if epoch % args.verbose == 0:
                evaluate()


user_field_M 6069
item_field_M 3953
field_M 10022
item_bind_M 3706
user_bind_M 6040
# of training: 994169
# of test: 6040
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


I0000 00:00:1743358464.030947    4793 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5717 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:08:00.0, compute capability: 8.9
I0000 00:00:1743358464.226767    4793 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled


Top 5 Recall: 0.0017, NDCG: 0.0008
Top 10 Recall: 0.0031, NDCG: 0.0013
Top 20 Recall: 0.0050, NDCG: 0.0017

Epoch 0
loss=-7565.3032, loss1=-7565.3032, reg=0.0000
Top 5 Recall: 0.0217, NDCG: 0.0129
Top 10 Recall: 0.0366, NDCG: 0.0177
Top 20 Recall: 0.0636, NDCG: 0.0245

Epoch 1
loss=-21172.8592, loss1=-21172.8592, reg=0.0000

Epoch 2
loss=-27130.6110, loss1=-27130.6110, reg=0.0000

Epoch 3
loss=-31822.7976, loss1=-31822.7976, reg=0.0000

Epoch 4
loss=-33914.0469, loss1=-33914.0469, reg=0.0000

Epoch 5
loss=-36346.6754, loss1=-36346.6754, reg=0.0000

Epoch 6
loss=-38072.1790, loss1=-38072.1790, reg=0.0000

Epoch 7
loss=-39862.7631, loss1=-39862.7631, reg=0.0000

Epoch 8
loss=-40701.2812, loss1=-40701.2812, reg=0.0000

Epoch 9
loss=-41417.4755, loss1=-41417.4755, reg=0.0000

Epoch 10
loss=-42032.4386, loss1=-42032.4386, reg=0.0000
Top 5 Recall: 0.0346, NDCG: 0.0216
Top 10 Recall: 0.0624, NDCG: 0.0306
Top 20 Recall: 0.1091, NDCG: 0.0423

Epoch 11
loss=-42932.2116, loss1=-42932.2116, reg=0.